In [ ]:
import random
hex_to_bin = {'0': "0000", '1': "0001", '2': "0010", '3': "0011", '4': "0100", '5': "0101", '6': "0110", '7': "0111",
                  '8': "1000", '9': "1001", 'a': "1010", 'b': "1011", 'c': "1100", 'd': "1101", 'e': "1110", 'f': "1111"}
bin_to_hex = {"0000": '0', "0001": '1', "0010": '2', "0011": '3', "0100": '4', "0101": '5', "0110": '6', "0111": '7',
              "1000": '8', "1001": '9', "1010": 'a', "1011": 'b', "1100": 'c', "1101": 'd', "1110": 'e', "1111": 'f'}

: 

In [75]:
def kcbingenerator():
    l = []
    for i in range(10):
        l.append(str(i))
    # print(l)
    for i in range(97, 103):
        l.append(chr(i))

    key = ""
    for i in range(32):
        a = random.randint(0, len(l)-1)
        key += l[a]

    l0 = key[:16]  # L0
    r0 = key[16:]  # R0

    binkey = ''  # Binary Cipher Key
    for i in r0:
        binkey += hex_to_bin[i]

    crr = ""
    crl = ""
    for i in binkey:
        if(i == '0'):
            crr += '1'
            crl += '1'
        else:
            crr += '0'
            crl += '1'

    Kcbin = binkey+crr  # Kc Encryption Key in Binary
    Ksbin = crl+crr  # Ks Decryption Key to be sent Binary
    Kc = ""  # Kc Hexadecimal
    for i in range(0, len(Kcbin), 4):
        b = Kcbin[i:i+4]
        Kc += bin_to_hex[b]

    Ks = ""  # Ks hexadecimal
    for i in range(0, len(Ksbin), 4):
        b = Ksbin[i:i+4]
        Ks += bin_to_hex[b]
    return Kcbin,Ksbin


In [76]:
def xor(a,b,c,d):
    r=''
    for i in range(32):
        r+=str(int(a[i])^int(b[i])^int(c[i])^int(d[i]))
    return r

In [77]:
def xor2(a,b):
    r=''
    for i in range(len(a)):
        r+=str(int(a[i])^int(b[i]))
    return r

In [78]:
def xnor(a,b):
    c = xor2(a,b)
    d=''
    for i in c:
        if(i=='0'):
            d+='1'
        else:
            d+='0'
    return d

In [79]:
def shiftrows(l):

    bigst = ''
    for i in range(0, 16, 4):
        temp = l[i]
        j = i+1
        st = ''
        while(j < i+4):
            st = st+l[j]
            j = j+1
        st = st+temp
        bigst = bigst+st
    return bigst

In [80]:
pi_Q = {0: 9, 1: 14, 2: 5, 3: 6, 4: 10, 5: 2, 6: 3, 7: 12,
        8: 15, 9: 0, 10: 4, 11: 13, 12: 7, 13: 11, 14: 1, 15: 8}
pi_P = {0: 3, 1: 15, 2: 14, 3: 0, 4: 5, 5: 4, 6: 11, 7: 12, 8: 13, 9: 10, 10: 9, 11: 6, 12: 7, 13: 8, 14: 2, 15: 1,
        }

bin_to_dec = {0: "0000", 1: "0001", 2: "0010", 3: "0011", 4: "0100", 5: "0101", 6: "0110", 7: "0111",
              8: "1000", 9: "1001", 10: "1010", 11: "1011", 12: "1100", 13: "1101", 14: "1110", 15: "1111"}

In [81]:
def substitutionQ(pt):
    re = ""
    k = int(pt, 2)
    re += bin_to_dec[pi_Q[k]]
    return re
def substitutionP(pt):
    re = ""
    k = int(pt, 2)
    re += bin_to_dec[pi_P[k]]
    return re

In [82]:
def Ffunction(p):
    for i in range(3):
        res=''
        for j in range(0,16,4):

            if(j%8==0):
                if(i!=1):
                    res+=substitutionP(p[j:j+4])
                else:
                    res+=substitutionQ(p[j:j+4])
            else:
                if(i!=1):
                    res+=substitutionQ(p[j:j+4])
                else:
                    res+=substitutionP(p[j:j+4])
       
        t1 = res[0:4]
        t2 = res[4:8]
        t3 = res[8:12]
        t4 = res[12:16]
        if(i!=2):
            T11 = t1[:2]+t2[:2]
            T21 = t1[2:]+t3[:2]
            T31 = t2[2:]+t4[:2]
            T41 = t3[2:]+t4[2:]
            p = T11+T21+T31+T41
        else:
            p = res
        
    return p

In [83]:
def railfence(text):
    key=2
    fence = [[None] * len(text) for i in range(key)]
    rails = list(range(key - 1)) + list(range(key - 1, 0, -1))

        # Write the characters in the fence pattern
    for i, char in enumerate(text):
        fence[rails[i % len(rails)]][i] = char

        # Read off the fence pattern
    result = []
    for rail in fence:
        result += [char for char in rail if char is not None]

    return ''.join(result)



In [84]:
def transposition(p):
    matrix = [[p[i+j*4] for i in range(4)] for j in range(4)]
    trans=''
    for i in range(4):
        l=''
        for j in range(4):
            l+=matrix[j][i]
        trans+=l
    return trans

In [85]:
def keygenerator(kc):
    keys = []
    for i in range(0, 128, 16):
        k = kc[i:i+16]
        keys.append(k)
    
    for i in range(len(keys)):
        keys[i] = shiftrows(keys[i])
    
    ffunced = []
    for i in keys:
        ffunced.append(Ffunction(i))
    
    tranrail=[]
    for i in range(8):
        if(i in [1,2,5,6]):
            tranrail.append(railfence(ffunced[i]))
        else:
            tranrail.append(transposition(ffunced[i]))
    FinalKeys=[]
    for i in range(0,8,2):
        l=''
        l=tranrail[i]+tranrail[i+1]
        FinalKeys.append(l)
    key5 = xor(FinalKeys[0],FinalKeys[1],FinalKeys[2],FinalKeys[3])
    FinalKeys.append(key5)
    return FinalKeys

In [198]:
def padding(binval):
    padded=[]
    if(len(binval)<128):
        x = 128-len(binval)
        newbinval = binval+'0'*x
        padded.append(newbinval)
    elif(len(binval)==128):
        padded.append(binval)
    else:
        if(len(binval)%128==0):
            for j in range(0,len(binval),128):
                t = binval[j:j+128]
                padded.append(t)
        else:
            z = len(binval)%128
            rem = 128-z
            binval = binval+'0'*rem
            for j in range(0,len(binval),128):
                t = binval[j:j+128]
                padded.append(t)
    return padded

In [86]:
def decryptionkey(x):
    l0 = x[:64]
    r0 = x[64:]
    k1 = xor2(l0,r0)
    dec_key = r0+k1
    return dec_key

In [87]:
x1,x2 = kcbingenerator()
print("Kc: ",x1)
print("Ks: ",x2)


Kc:  01110110100100001000101000011011101110110101001000100001101011001000100101101111011101011110010001000100101011011101111001010011
Ks:  11111111111111111111111111111111111111111111111111111111111111111000100101101111011101011110010001000100101011011101111001010011


In [88]:
dk = decryptionkey(x2)

In [89]:
dk

'10001001011011110111010111100100010001001010110111011110010100110111011010010000100010100001101110111011010100100010000110101100'

In [90]:
EncryptionRoundKeys = keygenerator(x1)

In [91]:
def decryptionkeygenerator(key):
    DecKeys = keygenerator(key)
    print(DecKeys)
    DecryptionRoundKeys = []
    DecryptionRoundKeys.append(DecKeys[4])
    DecryptionRoundKeys.append(DecKeys[0])
    DecryptionRoundKeys.append(DecKeys[3])
    DecryptionRoundKeys.append(DecKeys[2])
    DecryptionRoundKeys.append(DecKeys[1])
    return DecryptionRoundKeys

In [92]:
DecryptionRoundKeys = decryptionkeygenerator(dk)

['11100011011111100100001100001111', '11000001001100101110101111010101', '01110111010011000010100111001000', '01111010101001101001110110101011', '00101111101001100001110010111001']


In [93]:
for i in range(len(EncryptionRoundKeys)):
    print("Round Key ",i," : ",EncryptionRoundKeys[i])

Round Key  0  :  01110111010011000010100111001000
Round Key  1  :  01111010101001101001110110101011
Round Key  2  :  11100011011111100100001100001111
Round Key  3  :  11000001001100101110101111010101
Round Key  4  :  00101111101001100001110010111001


In [94]:
DecryptionRoundKeys

['00101111101001100001110010111001',
 '11100011011111100100001100001111',
 '01111010101001101001110110101011',
 '01110111010011000010100111001000',
 '11000001001100101110101111010101']

In [95]:
Deckey = []
Deckey.append(DecryptionRoundKeys[0])
Deckey.append(DecryptionRoundKeys[4])
Deckey.append(DecryptionRoundKeys[1])
Deckey.append(DecryptionRoundKeys[2])
Deckey.append(DecryptionRoundKeys[3])

In [96]:
for i in range(len(DecryptionRoundKeys)):
    print("Round Key ",i," : ",Deckey[i])

Round Key  0  :  00101111101001100001110010111001
Round Key  1  :  11000001001100101110101111010101
Round Key  2  :  11100011011111100100001100001111
Round Key  3  :  01111010101001101001110110101011
Round Key  4  :  01110111010011000010100111001000


In [101]:
Deckey[4]

'01110111010011000010100111001000'

In [207]:
def encryption(pt,j):
    P = []
    Cipher = ''
    for i in range(0,len(pt),32):
        P.append(pt[i:i+32])
    R011 = xnor(P[0],EncryptionRoundKeys[j])
    C1 = R011 #Cipher 1
    EFL1 = Ffunction(R011[:16])+Ffunction(R011[16:])
    Cipher+=R011
    R014 = xnor(P[3],EncryptionRoundKeys[j])
    C4 = R014 #Cipher 4
    EFR1 = Ffunction(R014[:16])+Ffunction(R014[16:])
    C2 = xor2(EFL1,P[2])
    Cipher+=C2
    C3 = xor2(EFR1,P[1])
    Cipher+=C3
    Cipher+=C4
    return Cipher

In [136]:
#listofencryption = encryption('01100100011010000110111101101110011010010010000001100110011010010110111001101001011100110110100001100101011100110010000001101111')

00100100110001101001010011011100


In [221]:
def decryption(ct):
    l = []
    for i in range(0,len(ct),32):
        l.append(ct[i:i+32])
    Plaintext=''
    P1 = xnor(l[0],Deckey[4])
    Plaintext+=P1
    DFL1 = Ffunction(l[0][:16])+Ffunction(l[0][16:])
    P3 = xor2(DFL1,l[1])
    P4 = xnor(l[3],Deckey[4])
    DFR1 = Ffunction(l[3][:16])+Ffunction(l[3][16:])
    P2 = xor2(DFR1,l[2])
    Plaintext+=P2
    Plaintext+=P3
    Plaintext+=P4
    return Plaintext
    

In [193]:
#y = decryption(listofencryption)

00100100110001101001010011011100


In [194]:
#y

['01100100011010000110111101101110',
 '01101001001000000110011001101001',
 '01101110011010010111001101101000',
 '01100101011100110010000001101111']

In [195]:
#ppp = ''
#for i in y:
 #   ppp+=i
#ppp

'01100100011010000110111101101110011010010010000001100110011010010110111001101001011100110110100001100101011100110010000001101111'

In [196]:
#a = xor2(ppp,'01100100011010000110111101101110011010010010000001100110011010010110111001101001011100110110100001100101011100110010000001101111')

In [197]:
#a

'00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'

In [191]:
#len('0000000000000000000000000000000000')

34

In [211]:
def Encryption(plaintext):
    Encrypted = ''
    x = plaintext.encode('utf-8')
    hexplaintext = x.hex()
    binplaintext=''
    for i in hexplaintext:
        binplaintext+=hex_to_bin[i]
    paddedlist = padding(binplaintext)
    for i in paddedlist:
        for j in range(1):
            txt = encryption(i,j)
        Encrypted+=txt
    return Encrypted

In [222]:
def Decryption(ciphertext):
    listofenc = []
    for i in range(0,len(ciphertext),128):
        listofenc.append(ciphertext[i:i+128])
    Plaintext = ''
    for i in listofenc:
        txt = decryption(i)
        Plaintext+=txt
    hexpt=''
    for i in range(0,len(Plaintext),4):
        hexpt+=bin_to_hex[Plaintext[i:i+4]]
    finalplain = bytes.fromhex(hexpt).decode('utf-8')
    return finalplain

In [218]:
etext = Encryption("dhoni finishes off in style india lifts the world cup after 28 years")

In [232]:
hexpt=''
for i in range(0,len(etext),4):
    hexpt+=bin_to_hex[etext[i:i+4]]
print(hexpt)
finalplain = bytes.fromhex(hexpt).decode('latin-1')
print(finalplain)
f = finalplain.encode('latin-1')
print(f)
l = f.hex()
print(l)

ecdbb9594aafe7b4d3bac550edc0f658eed5f65eeaa8c728de799719e1ddb25ee993ba5ee8d8814cda0b352cffdca45bec93b54250deed89808fc391ba8bf64eedd2a444b3c84ee6a5eb717688b3d637
ìÛ¹YJ¯ç´ÓºÅPíÀöXîÕö^ê¨Ç(ÞyáÝ²^éº^èØLÚ
5,ÿÜ¤[ìµBPÞíÃºöNíÒ¤D³ÈNæ¥ëqv³Ö7
b'\xec\xdb\xb9YJ\xaf\xe7\xb4\xd3\xba\xc5P\xed\xc0\xf6X\xee\xd5\xf6^\xea\xa8\xc7(\xdey\x97\x19\xe1\xdd\xb2^\xe9\x93\xba^\xe8\xd8\x81L\xda\x0b5,\xff\xdc\xa4[\xec\x93\xb5BP\xde\xed\x89\x80\x8f\xc3\x91\xba\x8b\xf6N\xed\xd2\xa4D\xb3\xc8N\xe6\xa5\xebqv\x88\xb3\xd67'
ecdbb9594aafe7b4d3bac550edc0f658eed5f65eeaa8c728de799719e1ddb25ee993ba5ee8d8814cda0b352cffdca45bec93b54250deed89808fc391ba8bf64eedd2a444b3c84ee6a5eb717688b3d637


In [219]:
len(etext)

640

In [223]:
ptext = Decryption(etext)

In [ ]:
ptext

: 